In [6]:
from mnist_config import ConfigDict
from mnist_network import MNISTNetwork
from data_provider import MNISTDataset
from absl import app, flags, logging
import random
import matplotlib.pyplot as plt
import os
import numpy as np
import tensorflow as tf

In [2]:
NUM_EPOCHS = 100
MOMENTUM = 0.0
DECAY_STEPS = 2000
DECAY_RATE = 0.96
BATCH_SIZE = 8
LOG_EVERY_STEPS = 50
SAVE_CHECKPOINT_EPOCHS = 5
EXPERIMENT_TYPE = "mnist"
DATA_DIR = "./mnist"   # ou "" pour téléchargement auto

In [3]:
from tensorflow.keras.models import load_model
model_path = "./models/xent_100epochs_lr0_01_generalization_batch8_model.keras"

model = load_model(
    model_path,
    compile=False,
    custom_objects={"MNISTNetwork": MNISTNetwork})

2026-01-26 20:25:51.624041: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [4]:
val_dataset_wrapper = MNISTDataset(
    data_dir=DATA_DIR,
    subset="test",   # ou "test" si c'est MNIST officiel
    batch_size=BATCH_SIZE,
    is_training=False
)

val_dataset = val_dataset_wrapper.dataset
num_examples = val_dataset_wrapper.num_examples
num_classes = val_dataset_wrapper.num_classes

In [7]:
margin_losses = []
accuracies = []
config = ConfigDict()

for images, labels in val_dataset:
    logits, endpoints = model(images, training=False)

    layers_list = [images]
    for i in range(len(config.filter_sizes_conv_layers)):
        layers_list.append(endpoints[f"conv_layer{i}"])

    one_hot_labels = tf.one_hot(labels, num_classes)

    acc = tf.reduce_mean(tf.cast(tf.argmax(logits, axis=1) == tf.cast(labels, tf.int64), tf.float32))
    accuracies.append(acc.numpy())

import numpy as np
#print("Validation Large Margin Loss:", np.mean(margin_losses))
print("Validation Accuracy:", np.mean(accuracies))

Validation Accuracy: 0.7687


2026-01-26 20:26:34.365499: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
